# Starter Drop-out Simulations

Jupyter notebook containing analysis for examining the impact of simulated starter drop-out on connectivity enrichment.

This notebook has some baseline calculations to model the impact of capture effiency on correctly IDing single starter networks. The final section simulates starter drop-out from actual data and reconstruct connectivity matrices based on newly called single starter networks.

Input for this notebook requires:
1) processed_barcodes_df, derived from 15_connectivity_analysis.ipynb
2) metadata_df.csv, derived from 15_connectivity_analysis.ipynb

Output includes:
1) Plots correpsonding to modeling of no starter vs. single starter vs. dual starter network calls as a function of starter drop-out
2) Plots related to connectivity and cell type proportions at a range of simulated cell capture effiencies

Module and their versions used when generating figures for the paper can be found in 'requirements.txt', which is stored in our GitHub repository: https://github.com/MEUrbanek/rabies_barcode_tech

This code was last amended by Maddie Urbanek on 12/16/2025

## Notebook set-up

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

In [1]:
#Activate operating system interfacing with JupyterLab
import os

In [2]:
#Change path name in function below to top-most directory containing data
os.chdir('/Users/maddieurbanek/Desktop/revision_data/resubmission/data/')

In [3]:
#For module versions, see requirements.txt on Github (LINK TO REFRESHED GITHUB)
import seaborn as sns
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import matplotlib.pyplot as plt
import ast
import networkx as nx
from scipy import stats
import scipy
import math
import time
import upsetplot
from matplotlib.ticker import PercentFormatter

plt.rc('font', family='Arial')

## Import data

In [ ]:
#Import processed_barcodes_df, which should contain UMI and SBARRO filtered barcodes
processed_barcodes_df=pd.read_table('./connectivity/processed_barcodes_df.csv')

In [ ]:
#Import mapped centroids as metadata_df
metadata_df=pd.read_csv('./connectivity/metadata_df.csv')

#Filter out low-score cells
metadata_df=metadata_df.loc[metadata_df['high_score'] >0.2]

#Amending column names
metadata_df = metadata_df.rename(columns={'type_updated': 'celltype', 'Unnamed: 0':'cellbarcode','dataset_id':'datasetid'})
metadata_df

## Modeling starter drop-out metrics

In [ ]:
#Make line plot showing how risk of calling single-starter that's actually dual-starter decreases as a function of helper capture
drop_out_risk=pd.DataFrame([.1,.2,.3,.4,.5,.6,.7,.8,.9,1])
drop_out_risk.columns=['capture']
drop_out_risk['both_observed']=drop_out_risk['capture']*drop_out_risk['capture']
drop_out_risk['none_observed']=(1-drop_out_risk['capture'])*(1-drop_out_risk['capture'])
drop_out_risk['one_observed']=1-drop_out_risk['none_observed']-drop_out_risk['both_observed']
drop_out_risk['risk_of_calling_one_not_two']=drop_out_risk['one_observed']/drop_out_risk['both_observed']

drop_out_risk

In [ ]:
#Plotting risk column
plt.plot(drop_out_risk['capture'],drop_out_risk['both_observed'],color='blue',label='Capture Both')
plt.plot(drop_out_risk['capture'],drop_out_risk['one_observed'],color='green',label='Capture One')
plt.plot(drop_out_risk['capture'],drop_out_risk['none_observed'],color='black',label='Capture None')
plt.legend()
plt.xlabel('% of Starters Capture')
plt.ylabel('False Single Starter Call/True Dual Starter Call')
plt.savefig('/Users/maddieurbanek/Desktop/revision_data/resubmission/figs/sfig_conn/starter_dropout/risk_dynamics.pdf')
plt.show() 

In [ ]:
#Plotting risk column
plt.plot(drop_out_risk['capture'],drop_out_risk['risk_of_calling_one_not_two'])
plt.axhline(y=1, color='black', linestyle=':')
plt.xlabel('% of Starters Capture')
plt.ylabel('False Single Starter Call/True Dual Starter Call')
plt.savefig('/Users/maddieurbanek/Desktop/revision_data/resubmission/figs/sfig_conn/starter_dropout/false_single_risk.pdf')
plt.show() 

## Starter drop-out simulations

In [ ]:
#Generate null matrix proportions
#Build null

#Subsetting metadata down to neuronal populations
subset_metadata_df = metadata_df[metadata_df.subclass != 'Microglia']
subset_metadata_df = subset_metadata_df[subset_metadata_df.subclass != 'Unknown']
subset_metadata_df = subset_metadata_df[subset_metadata_df.subclass != 'Vascular']
subset_metadata_df = subset_metadata_df[subset_metadata_df.subclass != 'Astrocyte']
subset_metadata_df = subset_metadata_df[subset_metadata_df.subclass != 'Oligo']
subset_metadata_df = subset_metadata_df[subset_metadata_df.subclass != 'IN-Mix-LAMP5']

uninfected_df = subset_metadata_df.loc[subset_metadata_df['datasetid'].isin(['u1','u2'])].groupby('subclass').count()['datasetid'].reset_index()
uninfected_df['proportion of celltype in nonstarter cells'] = uninfected_df['datasetid']/uninfected_df['datasetid'].sum()

#Rename 'datasetid' metadata column since it was overwritten by .count() function in preceding lines
uninfected_df.rename(columns = {'datasetid':'number of cells'},inplace = True)

#Clarify cell type assignments here should be used to generate non_starter null matrix populations
uninfected_df.rename(columns = {'subclass':'non_starter_cell_type'},inplace = True)
pooled_null = uninfected_df
pooled_null

In [ ]:
#Build function for randomly downsampling
from tqdm import tqdm

from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

def downsample_helpers(processed_barcodes_df, #unfiltered processed_barcodes_df
                       percent_to_take, #capture efficiency
                       bootstraps, #number of times to pull helpers and calculate average number of connections from
                       metadata_df,
                       pooled_null
                      ):

    #Pull list of putative helpers to choose from
    temp=processed_barcodes_df[processed_barcodes_df['helper'] == 'starter']
    starters_to_pull=int(temp['CBC'].nunique()*percent_to_take)
    print('Number of starters used to calculate connections:')
    print(starters_to_pull)

    bootstrapped_props=pd.DataFrame()
    bootstrapped_counts=pd.DataFrame()
    bootstrapped_enrichment_scores=pd.DataFrame()
    
    for n in tqdm(range(bootstraps)):
        #Get your starter pool
        starters=(np.random.choice(temp['CBC'].unique(),starters_to_pull,replace=False))
        #Pull all the cells from the processed_barcode_dataframe with the same barcodes as these helpers
        temp_1=temp[temp['CBC'].isin(starters)]
        to_pull=temp_1['barcode'].unique()
        downsampled_pbdf=processed_barcodes_df[processed_barcodes_df['barcode'].isin(to_pull)]

        df = downsampled_pbdf.loc[(downsampled_pbdf['single_starter_barcode'] == 'y') & (downsampled_pbdf['starter/nonstarter barcode'] == 'both')]
        cell_type_dict = dict(zip(df['CBC'],df['celltypes']))
        starter_df = df.loc[df['helper'] == 'starter'].groupby(['datasetid','CBC'])['barcode'].apply(set).reset_index()
        starter_dict = dict(zip(starter_df['CBC'],starter_df['barcode']))
        nonstarter_df = df.loc[df['helper'] == 'nonstarter'].groupby(['CBC'])['barcode'].apply(set).reset_index()
        nonstarter_dict = dict(zip(nonstarter_df['CBC'],nonstarter_df['barcode']))
        for starter in starter_dict.keys():
            matching_cells = []
            starter_barcode_set = starter_dict[starter]
            for nonstarter in nonstarter_dict.keys():
                nonstarter_barcode_set = nonstarter_dict[nonstarter]
                if len(starter_barcode_set.intersection(nonstarter_barcode_set))>=1:
                    matching_cells.append(nonstarter)
            starter_df.loc[starter_df['CBC'] == starter, 'matching_cells'] = str(matching_cells)
        
        #Process barcode data like normal
        starter_df['matching_cells'] = starter_df['matching_cells'].apply(ast.literal_eval)
        starter_df.rename(columns = {'CBC':'starter CBC', 'barcode':'starter barcodes', 'matching_cells':'non-starter CBC'}, inplace = True)
        starter_df['starter cell_type'] = starter_df['starter CBC'].apply(lambda x: cell_type_dict[x])
        conn_0_compiling = starter_df[['datasetid','starter cell_type','starter CBC','non-starter CBC']]
        conn_0_compiling['number of non-starters associated'] = conn_0_compiling['non-starter CBC'].apply(lambda x: len(x))
        conn_0_compiling['number of cells in single-starter network'] = conn_0_compiling['non-starter CBC'].apply(lambda x: len(x) +1)
        conn_1_compiling = starter_df.explode('non-starter CBC')
        conn_1_compiling['non-starter cell_type'] = conn_1_compiling['non-starter CBC'].apply(lambda x: cell_type_dict[x])
        conn_1_compiling = conn_1_compiling[['datasetid','starter cell_type','starter CBC','non-starter cell_type','non-starter CBC']]
        conn_1_compiling['conn_type'] = conn_1_compiling['non-starter cell_type'] +'+'+ conn_1_compiling['starter cell_type'] 
        conn_1_compiling['connection'] = conn_1_compiling.apply(lambda x: (x['non-starter CBC'], x['starter CBC'], {'w':1}), axis=1)
        feature_dict = dict(zip(metadata_df['cellbarcode'],metadata_df['broad_class']))
        conn_1_compiling['pre_broad_class'] = conn_1_compiling['non-starter CBC'].apply(lambda x: feature_dict[x])
        conn_1_compiling['post_subclass'] = conn_1_compiling['starter CBC'].apply(lambda x: feature_dict[x])
        feature_dict = dict(zip(metadata_df['cellbarcode'],metadata_df['subclass']))
        conn_1_compiling['pre_subclass'] = conn_1_compiling['non-starter CBC'].apply(lambda x: feature_dict[x])
        conn_1_compiling['post_subclass'] = conn_1_compiling['starter CBC'].apply(lambda x: feature_dict[x])
        conn_1_compiling['subclass_conn']=conn_1_compiling['pre_subclass'] + '+' + conn_1_compiling['post_subclass']
        subset_conn_1_compiling = conn_1_compiling[conn_1_compiling.post_subclass != 'Vascular']
        subset_conn_1_compiling = subset_conn_1_compiling[subset_conn_1_compiling.pre_subclass != 'Vascular']
        subset_conn_1_compiling = subset_conn_1_compiling[subset_conn_1_compiling.post_subclass != 'IN-Mix-LAMP5']
        subset_conn_1_compiling = subset_conn_1_compiling[subset_conn_1_compiling.pre_subclass != 'IN-Mix-LAMP5']
        subset_conn_1_compiling = subset_conn_1_compiling[subset_conn_1_compiling.post_subclass != 'Microglia']
        subset_conn_1_compiling = subset_conn_1_compiling[subset_conn_1_compiling.pre_subclass != 'Microglia']
        subset_conn_1_compiling = subset_conn_1_compiling[subset_conn_1_compiling.post_subclass != 'Astrocyte']
        subset_conn_1_compiling = subset_conn_1_compiling[subset_conn_1_compiling.pre_subclass != 'Astrocyte']
        subset_conn_1_compiling = subset_conn_1_compiling[subset_conn_1_compiling.post_subclass != 'Oligo']
        subset_conn_1_compiling = subset_conn_1_compiling[subset_conn_1_compiling.pre_subclass != 'Oligo']
        connection_counts=pd.DataFrame(subset_conn_1_compiling['subclass_conn'].value_counts())
        bootstrapped_counts = pd.concat([bootstrapped_counts,connection_counts],axis=1,sort=False)
        connection_counts.loc['total_counts'] = connection_counts.sum(numeric_only=True, axis=0)
        connection_props = connection_counts.divide(connection_counts.loc['total_counts'], axis=1)
        connection_props.drop(['total_counts'])
        bootstrapped_props = pd.concat([bootstrapped_props,connection_props],axis=1,sort=False)

        #Calculate average enrichment scores
        #Scaling null matrix
        starters = subset_conn_1_compiling.drop_duplicates(subset='starter CBC')
        starters.rename(columns={'starter cell_type': 'starter_cell_type'},inplace=True)
        feature_dict = dict(zip(metadata_df['cellbarcode'],metadata_df['subclass']))
        starters['starter_cell_type'] = starters['starter CBC'].apply(lambda x: feature_dict[x])
        starter_proportions=starters.groupby('starter_cell_type').count()['datasetid'].reset_index()
        starter_proportions['proportion of celltype in starter cells'] = starter_proportions['datasetid']/starter_proportions['datasetid'].sum()
        starter_proportions.rename(columns = {'datasetid':'number of cells'},inplace = True)
        starter_proportions.rename(columns = {'celltype':'starter_cell_type'},inplace = True)
        null_df = pooled_null.merge(starter_proportions, how='cross')
        null_df['proportion of connections'] = null_df['proportion of celltype in nonstarter cells']*null_df['proportion of celltype in starter cells']
        null_df['pre-post'] = null_df['non_starter_cell_type'] + "+" + null_df['starter_cell_type']
        
        #Now observed
        feature_dict = dict(zip(metadata_df['cellbarcode'],metadata_df['subclass']))
        subset_conn_1_compiling['starter cell_type'] = subset_conn_1_compiling['starter CBC'].apply(lambda x: feature_dict[x])
        subset_conn_1_compiling['non-starter cell_type'] = subset_conn_1_compiling['non-starter CBC'].apply(lambda x: feature_dict[x])
        conn_matrix_df = subset_conn_1_compiling.groupby(['non-starter cell_type','starter cell_type']).count()[['conn_type']].reset_index()
        conn_matrix_df.rename(columns = {'conn_type':'observed connections'}, inplace = True)
        conn_matrix_df['pre-post'] = conn_matrix_df['non-starter cell_type'] + '+' + conn_matrix_df['starter cell_type']
        missing_connections = list((set(null_df['pre-post'])).difference(set(conn_matrix_df['pre-post'])))
        df_new = pd.DataFrame({
            'pre-post':missing_connections,
            'observed connections':[0]*len(missing_connections),
            'non-starter cell_type': [x.split('+')[0] for x in missing_connections],
            'starter cell_type': [x.split('+')[1] for x in missing_connections],
        })
        conn_matrix_df = pd.concat([conn_matrix_df,df_new])
        conn_matrix_df['proportion of connections'] = conn_matrix_df['observed connections'] / conn_matrix_df['observed connections'].sum()
        
        #Calculate enrichment scores
        #Add any missing connections that aren't in the uninfected datasets 
        missing_connections = list((set(conn_matrix_df['pre-post'])).difference(set(null_df['pre-post'])))

        df_new = pd.DataFrame({
            'pre-post':missing_connections,
            'proportion of connections':[0]*len(missing_connections),   
        })

        null_df = pd.concat([null_df,df_new])
        null_dict = dict(zip(null_df['pre-post'],null_df['proportion of connections']))
        conn_matrix_df['null proportion of connections'] = conn_matrix_df['pre-post'].apply(lambda x: null_dict[x])
        conn_matrix_df['expected connections']=conn_matrix_df['observed connections'].sum() * conn_matrix_df['null proportion of connections']
        conn_matrix_df['expected connections']=conn_matrix_df['expected connections']+1
        conn_matrix_df['observed connections']=conn_matrix_df['observed connections']+1
        conn_matrix_df['ratio of proportion of connections/null'] = ((conn_matrix_df['observed connections']) / (conn_matrix_df['expected connections']))
        conn_matrix_df['log10 ratio of proportion of connections/null'] = (np.log10(conn_matrix_df['ratio of proportion of connections/null']))
        conn_matrix_df=conn_matrix_df.set_index('pre-post')
        enrichment_scores=conn_matrix_df[['log10 ratio of proportion of connections/null']]
        bootstrapped_enrichment_scores = pd.concat([bootstrapped_enrichment_scores,enrichment_scores],axis=1,sort=False)
    
    #Calculate average proportions across bootstraps
    bootstrapped_props=bootstrapped_props.fillna(0)
    average = pd.DataFrame(bootstrapped_props.mean(axis=1))
    bootstrapped_props['sem'] = bootstrapped_props.sem(axis=1)
    bootstrapped_props['average']=average

    bootstrapped_counts=bootstrapped_counts.fillna(0)
    average = pd.DataFrame(bootstrapped_counts.mean(axis=1))
    bootstrapped_counts['sem'] = bootstrapped_counts.sem(axis=1)
    bootstrapped_counts['average']=average

    bootstrapped_enrichment_scores=bootstrapped_enrichment_scores.fillna(0)
    average = pd.DataFrame(bootstrapped_enrichment_scores.mean(axis=1))
    bootstrapped_enrichment_scores['sem'] = bootstrapped_enrichment_scores.sem(axis=1)
    bootstrapped_enrichment_scores['average']=average


    #Export bootstrapped proportions
    bootstrapped_counts.to_csv(f'/Users/maddieurbanek/Desktop/revision_data/resubmission/data/connectivity/starter_drop_out_sim/{percent_to_take}_{bootstraps}_counts.csv')
    bootstrapped_props.to_csv(f'/Users/maddieurbanek/Desktop/revision_data/resubmission/data/connectivity/starter_drop_out_sim/{percent_to_take}_{bootstraps}_props.csv')
    bootstrapped_enrichment_scores.to_csv(f'/Users/maddieurbanek/Desktop/revision_data/resubmission/data/connectivity/starter_drop_out_sim/{percent_to_take}_{bootstraps}_enrichment.csv')
    
    return bootstrapped_props

In [ ]:
#Run simulation for each capture efficiency rate, 50 iterations
simulation=downsample_helpers(processed_barcodes_df,
                   1,
                   50, metadata_df, pooled_null)

simulation=downsample_helpers(processed_barcodes_df,
                   0.9,
                   50, metadata_df, pooled_null)

simulation=downsample_helpers(processed_barcodes_df,
                   0.8,
                   50, metadata_df, pooled_null)

simulation=downsample_helpers(processed_barcodes_df,
                   0.7,
                   50, metadata_df, pooled_null)

simulation=downsample_helpers(processed_barcodes_df,
                   0.6,
                   50, metadata_df, pooled_null)

simulation=downsample_helpers(processed_barcodes_df,
                   0.5,
                   50, metadata_df, pooled_null)

simulation=downsample_helpers(processed_barcodes_df,
                   0.4,
                   50, metadata_df, pooled_null)

simulation=downsample_helpers(processed_barcodes_df,
                   0.3,
                   50, metadata_df, pooled_null)

simulation=downsample_helpers(processed_barcodes_df,
                   0.2,
                   50, metadata_df, pooled_null)

simulation=downsample_helpers(processed_barcodes_df,
                   0.1,
                   50, metadata_df, pooled_null)

In [ ]:
#Read in previous simulations
temp=pd.DataFrame()

sim_list=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
for i in sim_list:
    sim=pd.read_table(f'/Users/maddieurbanek/Desktop/revision_data/resubmission/data/connectivity/starter_drop_out_sim/{i}_50_props.csv',delimiter=',')
    sim=sim.set_index('subclass_conn')
    sim=sim[['average','sem']]
    sim.columns=[f'average_{i}',f'sem_{i}']
    temp = pd.concat([temp,sim],axis=1,sort=False)

temp

In [ ]:
#Split pooled dataframe into starter-type specific
temp['subclass_connections']=temp.index
temp

temp[['pre', 'starter']] = temp['subclass_connections'].str.split('+', expand=True)

In [ ]:
cr=temp[temp['starter'] == 'Cajal-Retzius cell']
imm=temp[temp['starter'] == 'EN-Immature']
l23=temp[temp['starter'] == 'EN-L2_3-IT']
l4=temp[temp['starter'] == 'EN-L4-IT']
dl=temp[temp['starter'] == 'EN-Deep Layer']
mge=temp[temp['starter'] == 'IN-MGE']
cge=temp[temp['starter'] == 'IN-CGE']
dge=temp[temp['starter'] == 'IN-DGE']
ipc=temp[temp['starter'] == 'IPC']

In [ ]:
#Make function for plotting proportions of connections for each cell type
def starter_sim_formatting(dataset,celltype):

    plt.rc('font', family='Arial', size=14)
    
    dataset=dataset.drop(columns=['subclass_connections', 'starter'])
    dataset=dataset.set_index('pre')
    averages=dataset[['average_0.1','average_0.2','average_0.3','average_0.4','average_0.5','average_0.6','average_0.7','average_0.8','average_0.9','average_1']]
    averages.columns=[.1,.2,.3,.4,.5,.6,.7,.8,.9,1]
    transposed=averages.T

    sem=dataset[['sem_0.1','sem_0.2','sem_0.3','sem_0.4','sem_0.5','sem_0.6','sem_0.7','sem_0.8','sem_0.9','sem_1']]
    sem.columns=[.1,.2,.3,.4,.5,.6,.7,.8,.9,1]
    t_sem=sem.T
    t_sem

    upper = transposed + t_sem
    lower = transposed - t_sem

    print(upper)
    
    #ax = lineplot(data=transposed, x=transposed.index, y="Cajal-Retzius cell", errorbar=None,color='#d6616b')
    #ax.fill_between(transposed.index, lower['Cajal-Retzius cell'], upper["Cajal-Retzius cell"], alpha=0.2,color='#d6616b')

    ax = lineplot(data=transposed, x=transposed.index, y="EN-L2_3-IT", errorbar=None,color='#e7ba52')
    ax.fill_between(transposed.index, lower['EN-L2_3-IT'], upper["EN-L2_3-IT"], alpha=0.2,color='#e7ba52')

    ax = lineplot(data=transposed, x=transposed.index, y="EN-Deep Layer", errorbar=None,color='#843c39')
    ax.fill_between(transposed.index, lower['EN-Deep Layer'], upper["EN-Deep Layer"], alpha=0.2,color='#843c39')

    ax = lineplot(data=transposed, x=transposed.index, y="EN-L4-IT", errorbar=None,color='#e7cb94')
    ax.fill_between(transposed.index, lower['EN-L4-IT'], upper["EN-L4-IT"], alpha=0.2,color='#e7cb94')

    ax = lineplot(data=transposed, x=transposed.index, y="EN-Immature", errorbar=None,color='#8c6d31')
    ax.fill_between(transposed.index, lower['EN-Immature'], upper["EN-Immature"], alpha=0.2,color='#8c6d31')

    #ax = lineplot(data=transposed, x=transposed.index, y="IN-MGE", errorbar=None,color='#a55194')
    #ax.fill_between(transposed.index, lower['IN-MGE'], upper["IN-MGE"], alpha=0.2,color='#a55194')

    ax = lineplot(data=transposed, x=transposed.index, y="IN-CGE", errorbar=None,color='#e7969c')
    ax.fill_between(transposed.index, lower['IN-CGE'], upper["IN-CGE"], alpha=0.2,color='#e7969c')

    #ax = lineplot(data=transposed, x=transposed.index, y="RG", errorbar=None,color='#8ca252')
    #ax.fill_between(transposed.index, lower['RG'], upper["RG"], alpha=0.2,color='#8ca252')

    #ax = lineplot(data=transposed, x=transposed.index, y="IPC", errorbar=None,color='#cedb9c')
    #ax.fill_between(transposed.index, lower['IPC'], upper["IPC"], alpha=0.2,color='#cedb9c')

    ax = lineplot(data=transposed, x=transposed.index, y="IN-DGE", errorbar=None,color='#ce6dbd')
    ax.fill_between(transposed.index, lower['IN-DGE'], upper["IN-DGE"], alpha=0.2,color='#ce6dbd')

    ax.set_xlabel('Capture Efficiency')
    ax.set_ylabel('Proportion of Connection IDed')

    plt.savefig(f'/Users/maddieurbanek/Desktop/revision_data/resubmission/figs/sfig_conn/starter_dropout/{celltype}.pdf') 
    plt.show()
    #return transposed, upper, lower

In [ ]:
starter_sim_formatting(cr,'cr')

In [ ]:
starter_sim_formatting(imm,'imm')

In [ ]:
starter_sim_formatting(l23,'l23')

In [ ]:
starter_sim_formatting(l4,'l4')

In [ ]:
starter_sim_formatting(dl,'dl')

In [ ]:
starter_sim_formatting(mge,'mge')

In [ ]:
starter_sim_formatting(cge,'cge')

In [ ]:
starter_sim_formatting(dge,'dge')

In [ ]:
starter_sim_formatting(ipc,'ipc')

In [ ]:
#Number of connections per capture efficiency
#Read in previous simulations
average_list=[]
sem_list=[]

sim_list=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
for i in sim_list:
    sim=pd.read_table(f'/Users/maddieurbanek/Desktop/revision_data/resubmission/data/connectivity/starter_drop_out_sim/{i}_50_counts.csv',delimiter=',')
    sim=sim.drop(columns=['sem', 'average','subclass_conn'])
    sim.loc['total_connections'] = sim.sum(numeric_only=True, axis=0)
    total_connections=sim.loc['total_connections']
    average=total_connections.mean()
    sem=total_connections.sem()
    average_list.append(average)
    sem_list.append(sem)

average_list
average_conn = pd.DataFrame(list(zip(sim_list, average_list)), columns=['efficiency', 'average'])
sem_conn = pd.DataFrame(list(zip(sim_list, sem_list)), columns=['efficiency', 'sem'])
sem_conn

In [ ]:
#Plot
ax = lineplot(data=average_conn, x="efficiency", y="average", errorbar=None,color='black')
ax.fill_between(average_conn['efficiency'], average_conn['average']-sem_conn['sem'], average_conn['average']+sem_conn['sem'],color='black', alpha=0.30)

ax.set_xlabel('Capture Efficiency')
ax.set_ylabel('Number of Connections IDed')

plt.savefig(f'/Users/maddieurbanek/Desktop/revision_data/resubmission/figs/sfig_conn/starter_dropout/average_number_connections.pdf') 
plt.show()

In [ ]:
#Make connectivity enrichment heatmaps with different capture effiencies

In [ ]:
#Build function for plotting connectivity enrichment matrices
def downsampled_conn(dataset):

    #get counts+SEM
    counts=pd.read_table(f'/Users/maddieurbanek/Desktop/revision_data/resubmission/data/connectivity/starter_drop_out_sim/{dataset}_50_counts.csv',delimiter=',')
    counts=counts.fillna(0)
    counts=counts.round(1)
    counts['average'] = counts['average'].map(str)
    counts['sem'] = counts['sem'].map(str)
    counts['average_sem']=counts['average']+'±'+counts['sem']
    temp=counts[['subclass_conn','average_sem']]
    temp[['non-starter cell_type','starter cell_type']] = temp['subclass_conn'].str.split('+', expand=True)
    temp
    pivot2 = temp.pivot(index='non-starter cell_type', columns='starter cell_type')['average_sem']
    pivot2=pivot2.fillna(0) 
    pivot2

    #get enrichment score
    enrichment=pd.read_table(f'/Users/maddieurbanek/Desktop/revision_data/resubmission/data/connectivity/starter_drop_out_sim/{dataset}_50_enrichment.csv',delimiter=',')
    temp=enrichment[['pre-post','average']]
    temp[['non-starter cell_type','starter cell_type']] = temp['pre-post'].str.split('+', expand=True)
    temp
    pivot = temp.pivot(index='non-starter cell_type', columns='starter cell_type')['average']
    pivot=pivot.fillna(0) 

    #plot
    #Format input for heatmap plot
    
    pivot = pivot.reindex(['RG','IPC','EN-Immature','EN-L2_3-IT','EN-L4-IT','EN-Deep Layer','Cajal-Retzius cell','IN-CGE','IN-MGE','IN-DGE'], level=0) \
    .T.reindex(['RG','IPC','EN-Immature','EN-L2_3-IT','EN-L4-IT','EN-Deep Layer','Cajal-Retzius cell','IN-CGE','IN-MGE','IN-DGE']).T

    pivot2 = pivot2.reindex(['RG','IPC','EN-Immature','EN-L2_3-IT','EN-L4-IT','EN-Deep Layer','Cajal-Retzius cell','IN-CGE','IN-MGE','IN-DGE'], level=0) \
    .T.reindex(['RG','IPC','EN-Immature','EN-L2_3-IT','EN-L4-IT','EN-Deep Layer','Cajal-Retzius cell','IN-CGE','IN-MGE','IN-DGE']).T

    plt.figure(figsize=(15,8))

    #Scaling is for log10 transformation
    sns.heatmap(pivot,  annot= pivot2, fmt = '', 
                square = True, 
                vmin=-2.0, 
                vmax =2,
                center=0,
               cmap = 'bwr')
    plt.ylabel('Non-starter')
    plt.xlabel('Starter')
    plt.rcParams.update({'font.size': 8})

    #Save figure as an .svg file
    plt.savefig(f'/Users/maddieurbanek/Desktop/revision_data/resubmission/figs/sfig_conn/starter_dropout/conn_matrices/{dataset}_matrix.svg', bbox_inches = 'tight', format = 'svg')
    

In [ ]:
#Run downsampled connectivity
downsampled_conn(1)
downsampled_conn(0.9)
downsampled_conn(0.8)
downsampled_conn(0.7)
downsampled_conn(0.6)
downsampled_conn(0.5)
downsampled_conn(0.4)
downsampled_conn(0.3)
downsampled_conn(0.2)
downsampled_conn(0.1)

### If needing to reload saved simulation

In [ ]:
counts=pd.read_table('/Users/maddieurbanek/Desktop/revision_data/resubmission/data/connectivity/starter_drop_out_sim/0.1_50_counts.csv',delimiter=',')
counts=counts.fillna(0)
counts=counts.round(1)

counts['average'] = counts['average'].map(str)
counts['sem'] = counts['sem'].map(str)
counts['average_sem']=counts['average']+'±'+counts['sem']


temp=counts[['subclass_conn','average_sem']]
temp[['non-starter cell_type','starter cell_type']] = temp['subclass_conn'].str.split('+', expand=True)
temp

pivot2 = temp.pivot(index='non-starter cell_type', columns='starter cell_type')['average_sem']
pivot2=pivot2.fillna(0) 
pivot2

In [ ]:
enrichment=pd.read_table('/Users/maddieurbanek/Desktop/revision_data/resubmission/data/connectivity/starter_drop_out_sim/0.1_50_enrichment.csv',delimiter=',')
enrichment

In [ ]:
temp=enrichment[['pre-post','average']]
temp[['non-starter cell_type','starter cell_type']] = temp['pre-post'].str.split('+', expand=True)
temp

pivot = temp.pivot(index='non-starter cell_type', columns='starter cell_type')['average']
pivot 

In [ ]:
#Format input for heatmap plot
    pivot = observed.pivot(index='non-starter cell_type', columns='starter cell_type')['log10 ratio of proportion of connections/null']
    
    observed['real observed connections']=observed['post_hoc_significance'] + (observed['observed connections']-1).astype(str)
    pivot2=observed.pivot(index='non-starter cell_type', columns='starter cell_type')['real observed connections']

    pivot = pivot.reindex(['RG','IPC','EN-Immature','EN-L2_3-IT','EN-L4-IT','EN-Deep Layer','Cajal-Retzius cell','IN-CGE','IN-MGE','IN-DGE'], level=0) \
    .T.reindex(['RG','IPC','EN-Immature','EN-L2_3-IT','EN-L4-IT','EN-Deep Layer','Cajal-Retzius cell','IN-CGE','IN-MGE','IN-DGE']).T

    pivot2 = pivot2.reindex(['RG','IPC','EN-Immature','EN-L2_3-IT','EN-L4-IT','EN-Deep Layer','Cajal-Retzius cell','IN-CGE','IN-MGE','IN-DGE'], level=0) \
    .T.reindex(['RG','IPC','EN-Immature','EN-L2_3-IT','EN-L4-IT','EN-Deep Layer','Cajal-Retzius cell','IN-CGE','IN-MGE','IN-DGE']).T

    observed['expected connections']=observed['expected connections']+1
    observed['observed connections']=observed['observed connections']+1
    observed.to_csv(f'/Users/maddieurbanek/Desktop/revision_data/resubmission/figs/sfig_conn/starter_dropout/conn_matrices/{dataset_id}_observation_table.csv')
    
    plt.figure(figsize=(15,8))

    #Scaling is for log10 transformation
    sns.heatmap(pivot,  annot= pivot2, fmt = '', 
                square = True, 
                vmin=-2.0, 
                vmax =2,
                center=0,
               cmap = 'bwr')
    plt.ylabel('Non-starter')
    plt.xlabel('Starter')

    #Save figure as an .svg file
    plt.savefig(f'/Users/maddieurbanek/Desktop/revision_data/resubmission/figs/sfig_conn/starter_dropout/conn_matrices/{dataset_id}_matrix.svg', bbox_inches = 'tight', format = 'svg')

    print('Observation table:')
    print(observed[['pre-post','observed connections']])
    print()
    print('Total connections:')
    print(observed['observed connections'].sum())
    
    return observed